## Libraries (local machine)

In [6]:
from datetime import datetime, timedelta
from dotenv import load_dotenv
from os import getenv
from random import randint
import numpy as np
import pandas as pd

load_dotenv()

True

## Libraries (Google Colab)

In [ ]:
from datetime import datetime, timedelta
from google.colab import userdata
from random import randint
import numpy as np
import pandas as pd

def getenv(secretName: str, default_value):
  try:
    return userdata.get(secretName)
  except:
    return default_value

## Classes definition

In [ ]:
from ast import match_case
#from typing import Any, NewType

class Project:
   def __init__(self, name:str, DF_date: datetime, MCS_date: datetime, Pilot_date: datetime, SOP_date: datetime):
      self.name = name
      self.important_dates = {
         "Design freeze": DF_date,
         "MCS": MCS_date,
         "Pilot": Pilot_date,
         "SOP": SOP_date
      }

   def __str__(self):
      return self.name

class Item_Master:
   def __init__(self):
      columns = {
        # ECN identification data
        "Project": [],
        "ECN": [],
        "ECN release": [],
        "RFQ date": [],
        # Part number data
        "Part number": [],
        "Complexity": [],
        "EAU": [],  # EAU stands for Estimated Annual Use
        # Supplier data
        "Supplier ID": [],
        "Supplier name": [],
        "Quotation date": [],
        "Price": [],
        "Lead time": [],
        # Sample delivery data
        "ETA": [],
        "Delivery date": [],
        "ISIR documents": [],
        # Environment data
        "REQ date": [],
        "PO date": [],
        "ISIR approval": [],
        "PPAP approval": [],
        "Contract date": [],
        # Fuzzy inputs
        "Quotation time": [],
        "OTD": [],
        "Delivery time": [],
        # Other information
        "FY Spend": [], # EAU * Price
        "Awarded": [] # bool (False only for quotations, True for awarded business)
      }
    
      self.df = pd.DataFrame(columns)

class Part_Number:
    def __init__(self, pn: str, complexity: str, eau: int):
        self.pn = pn
        self.complexity = complexity
        self.eau = eau  # I am using an integer because most of the materials have EA as UOM, with limited exceptions.

    def __str__(self):
        return self.pn

class ECN:
    def __init__(self, project: Project, ecn_id: str, ecn_date: datetime, pn_list: list[Part_Number]):
        self.project = project
        self.ecn_id = ecn_id
        self.ecn_date = ecn_date
        self.items = pn_list
        self.quotations = []

    def __str__(self):
        return self.ecn_id

class Quotation:
    def __init__(self, ecn: ECN, supplier: object):
        self.ecn = ecn
        self.supplier = supplier

        self.ecn.quotations.append(self)

        columns = {
        # ECN identification data
        "Project": [],
        "ECN": [],
        "ECN release": [],
        "RFQ date": [],
        # Part number data
        "Part number": [],
        "Complexity": [],
        "EAU": [],  # EAU stands for Estimated Annual Use
        # Supplier data
        "Supplier ID": [],
        "Supplier name": [],
        "Quotation date": [],
        "Price": [],
        "Lead time": [],
        # Other information
        "FY Spend": [], # EAU * Price
        "Awarded": [] # bool (False only for quotations, True for awarded business)
        }

        self.df = pd.DataFrame(columns)

class Supplier:
    def __init__(self, id: str | int, name: str, price_profile: str = "regular", quotation_profile: str = "regular", sample_delivery_punctuality_profile: str = "regular"):
        self.id = self.__check_id(id)
        self.name = name
        self.quotations = []

        price_profile_map = { # This is a factor to multiply; average and standard deviation
          "low": (0.85, 0.85),
          "regular": (1, 1),
          "high": (1.2, 1.1)
        }

        quotation_profile_map = { # This is a factor to multiply; average and standard deviation
          "low": (28.975, 25.1133753461483),
          "regular": (27.7241379310345, 21.5974276436511),
          "high": (24.9444444444444, 10.258266234788)
        }

        punctuality_profile_map = { # Probability
          "low": 0.19047619047619,
          "regular": 0.473684210526316,
          "high": 0.638888888888889
        }

        self.ETA_difference = {
            "punctual": (0.888888888888889, 1.01273936708367),
            "unpunctual": (4.24137931034483, 2.69463981708917)
        }

        µ_price_profile_factor, σ_price_profile_factor = price_profile_map[price_profile]

        self.price_complexity_map = {
            "low": (float(getenv("AVG_PRICE_HIGH_COMPLEXITY", 0)) * µ_price_profile_factor, float(getenv("STDEV_PRICE_HIGH_COMPLEXITY", 1)) * σ_price_profile_factor),
            "medium": (float(getenv("AVG_PRICE_MEDIUM_COMPLEXITY", 0)) * µ_price_profile_factor, float(getenv("STDEV_PRICE_MEDIUM_COMPLEXITY", 1)) * σ_price_profile_factor),
            "high": (float(getenv("AVG_PRICE_LOW_COMPLEXITY", 0)) * µ_price_profile_factor, float(getenv("STDEV_PRICE_LOW_COMPLEXITY", 1)) * σ_price_profile_factor),
            "minimum": float(getenv("MINIMUM_PRICE", 0)) * µ_price_profile_factor
        }

        self.µ_quotation_time, self.σ_quotation_time = quotation_profile_map[quotation_profile]
        self.minimum_quotation_time = 9

        self.µ_delivery_time = 34.6206896551724
        self.σ_delivery_time = 16.2802512871323

        self.µ_isir_documents_upload = 0.348484848484849
        self.σ_isir_documents_upload = 0.936317241478537

        self.punctual_p = punctuality_profile_map[sample_delivery_punctuality_profile]

    def __str__(self):
        return self.name

    def __check_id(self, id: str | int):
        if len(str(id)) < 8 or len(str(id)) > 8:
          raise Exception("Invalid supplier ID")
        else:
          return str(id)

    def quote(self, ecn: ECN, delivery_time: int = 0, random_rfq_date: bool = True):
        for quotation in self.quotations:
            if quotation.ecn.ecn_id == ecn.ecn_id:
                raise Exception(f"{self.name} already quoted {ecn.ecn_id}.")

        quotation = Quotation(ecn, self)

        min_price = self.price_complexity_map["minimum"]

        if random_rfq_date:
          rfq_date = datetime.today() - timedelta(days=randint(40, 60))
        else:
          rfq_date = datetime.today()

        quotation_time = max(round(np.random.normal(self.µ_quotation_time, self.σ_quotation_time)), self.minimum_quotation_time)
        quotation_date = rfq_date + timedelta(days=quotation_time)

        for part_number in ecn.items:
            complexity = part_number.complexity
            µ, σ = self.price_complexity_map[complexity]
            price = round(max(np.random.normal(µ, σ), min_price), 2)

            if delivery_time == 0:
                lt = np.nan
            elif delivery_time > 0:
                lt = delivery_time
            else:
                raise Exception("Lead time cannot be less than 1 day.")
            
            quotation.df.loc[-1] = [ecn.project.name, ecn.ecn_id, ecn.ecn_date, rfq_date, part_number.pn, complexity, part_number.eau, self.id, self.name, quotation_date, price, lt, part_number.eau*lt, False]

        self.quotations.append(quotation)
        return quotation

    def sell(self, reference: ECN|Quotation):
      if isinstance(reference, ECN):
        for quotation in self.quotations:
          if quotation.ecn == reference:
            reference = quotation

      if isinstance(reference, Quotation):
        for i in range(len(reference.price_list[next(iter(reference.price_list))])):
           pass
      else:
        raise Exception("Invalid reference.")

In [5]:
example = {
    "list 1": [1,2,3],
    "list 2": [4,5,6]
}

len(example[next(iter(example))])

3

In [ ]:
class Environment:
  def __init__(self):
    self.suppliers = []
    self.ecns = []

    self.item_master = Item_Master().df
    
    self.part_kinds = {
        "A": {
            "average": float(getenv("AVG_A_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_A_PART_KIND", 1)),
            "complexity": {"low": 0.6818181818182, "medium": 0.318181818181818, "high": 0},
            "parts": []
        },
        "B": {
            "average": float(getenv("AVG_B_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_B_PART_KIND", 1)),
            "complexity": {"low": 1/3, "medium": 2/3, "high": 0},
            "parts": []
        },
        "C": {
            "average": float(getenv("AVG_C_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_C_PART_KIND", 1)),
            "complexity": {"low": 1, "medium": 0, "high": 0},
            "parts": []
        },
        "D": {
            "average": float(getenv("AVG_D_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_D_PART_KIND", 1)),
            "complexity": {"low": 0.090909090909090909, "medium": 0.727272727272727, "high": 0.181818181818182},
            "parts": []
        },
        "E": {
            "average": float(getenv("AVG_E_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_E_PART_KIND", 1)),
            "complexity": {"low": 0, "medium": 0, "high": 1},
            "parts": []
        },
        "F": {
            "average": float(getenv("AVG_F_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_F_PART_KIND", 1)),
            "complexity": {"low": 0, "medium": 0, "high": 1},
            "parts": []
        },
        "G": {
            "average": float(getenv("AVG_G_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_G_PART_KIND", 1)),
            "complexity": {"low": 0, "medium": 0, "high": 1},
            "parts": []
        }
    }

  def add_supplier(self, supplier: Supplier):
    self.suppliers.append(supplier)

  def add_suppliers(self, suppliers: list[Supplier]):
    for supplier in suppliers:
      self.add_supplier(supplier)

  def gen_ecn(self, project: Project, qty: int):
    for i in range(qty):
      ecn_part_numbers = []

      while len(ecn_part_numbers) == 0:
        for key in self.part_kinds.keys():
          kind_complexity_keys = list(self.part_kinds[key]["complexity"].keys())
          kind_complexity_probabilities = list(self.part_kinds[key]["complexity"].values())

          for j in range(max(int(np.random.normal(self.part_kinds[key]["average"], self.part_kinds[key]["stdev"])), 0)):
            category_part_number = len(self.part_kinds[key]["parts"]) + 1
            complexity = np.random.choice(kind_complexity_keys, p=kind_complexity_probabilities)

            part_number = Part_Number(pn=f"A0{key}{str(category_part_number).zfill(6)}", complexity=complexity, eau=100)

            self.part_kinds[key]["parts"].append(part_number)
            ecn_part_numbers.append(part_number)

      ecn_number = len(self.ecns) + 1
      self.ecns.append(ECN(project=project, ecn_id=f"ECN{str(ecn_number).zfill(7)}", ecn_date=datetime.date, pn_list=ecn_part_numbers))

  def gen_initial_quotation_df(self, project: Project, ecn_qty: int):
    self.gen_ecn(project, ecn_qty)

    for ecn in self.ecns:
      for supplier in self.suppliers:
        supplier.quote(ecn)
        for quotation in supplier.quotations:
          self.item_master = pd.concat([self.item_master, quotation.df], ignore_index=True)

    return self.item_master

In [20]:
env = Environment()

supplier_1 = Supplier(id="10000001", name="Tuberías ABC, S.A. de C.V.", price_profile="regular", quotation_profile="high", sample_delivery_punctuality_profile="regular")
supplier_2 = Supplier(id="10000002", name="Tuberías DEF, S.A.", price_profile="regular", quotation_profile="regular", sample_delivery_punctuality_profile="regular")
supplier_3 = Supplier(id="10000003", name="Tuberías GHI, S.A.S.", price_profile="high", quotation_profile="low", sample_delivery_punctuality_profile="regular")
supplier_4 = Supplier(id="10000004", name="Tuberías JKL y asociados, S. en C.", price_profile="high", quotation_profile="low", sample_delivery_punctuality_profile="low")
supplier_5 = Supplier(id="10000005", name="Tuberías MNO, S.A.P.I. de C.V.", price_profile="low", quotation_profile="low", sample_delivery_punctuality_profile="high")

env.add_suppliers([supplier_1, supplier_2, supplier_3, supplier_4, supplier_5])

In [21]:
df = env.gen_initial_quotation_df(10)
df

,ECN,ECN release,RFQ date,Part number,Complexity,Supplier ID,Supplier name,Quotation date,Price,Lead time,ETA,Delivery date,ISIR documents,Quotation time,OTD,Awarded
0,ECN0000001,<method 'date' of 'datetime.datetime' objects>,2025-07-01,A0D000001,medium,10000001,"Tuberías ABC, S.A. de C.V.",2025-07-19,97.87,NaN,NaN,NaN,NaN,18.0,NaN,0.0
1,ECN0000001,<method 'date' of 'datetime.datetime' objects>,2025-07-01,A0E000001,high,10000001,"Tuberías ABC, S.A. de C.V.",2025-07-19,19.33,NaN,NaN,NaN,NaN,18.0,NaN,0.0
0,ECN0000001,<method 'date' of 'datetime.datetime' objects>,2025-07-11,A0D000001,medium,10000002,"Tuberías DEF, S.A.",2025-07-20,0.00,NaN,NaN,NaN,NaN,9.0,NaN,False
1,ECN0000001,<method 'date' of 'datetime.datetime' objects>,2025-07-11,A0E000001,high,10000002,"Tuberías DEF, S.A.",2025-07-20,0.00,NaN,NaN,NaN,NaN,9.0,NaN,False
0,ECN0000001,<method 'date' of 'datetime.datetime' objects>,2025-07-14,A0D000001,medium,10000003,"Tuberías GHI, S.A.S.",2025-07-26,0.00,NaN,NaN,NaN,NaN,12.0,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,ECN0000010,<method 'date' of 'datetime.datetime' objects>,2025-07-07,A0E000010,high,10000005,"Tuberías MNO, S.A.P.I. de C.V.",2025-07-16,10.57,NaN,NaN,NaN,NaN,9.0,NaN,False
2,ECN0000010,<method 'date' of 'datetime.datetime' objects>,2025-07-07,A0E000011,high,10000005,"Tuberías MNO, S.A.P.I. de C.V.",2025-07-16,12.48,NaN,NaN,NaN,NaN,9.0,NaN,False
3,ECN0000010,<method 'date' of 'datetime.datetime' objects>,2025-07-07,A0E000012,high,10000005,"Tuberías MNO, S.A.P.I. de C.V.",2025-07-16,17.22,NaN,NaN,NaN,NaN,9.0,NaN,False
4,ECN0000010,<method 'date' of 'datetime.datetime' objects>,2025-07-07,A0E000013,high,10000005,"Tuberías MNO, S.A.P.I. de C.V.",2025-07-16,10.66,NaN,NaN,NaN,NaN,9.0,NaN,False


In [22]:
df.to_excel("report.xlsx")